In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import random

In [ ]:
# Function to preprocess images and extract features
def preprocess_images(directory, condition, target_width, target_height):
    images = []
    labels = []
    for filename in os.listdir(directory):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(directory, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (target_width, target_height))
            images.append(img.flatten())  # Flatten the image and append to the list
            if condition == 'normal':
                labels.append(0)
            elif condition == 'atelectasis':
                labels.append(1)
            elif condition == 'pneumonia':
                labels.append(2)
            else:
                raise ValueError("Invalid condition specified")

    return images, labels

In [ ]:
# Specify the directories for each condition
normal_dir = 'Path to Normal file'
atelectasis_dir = 'Path to Atelectasis file'
pneumonia_dir = 'Path to Pneumonia file'

In [ ]:
# Specify the target dimensions for resizing
target_width = 100
target_height = 100

In [ ]:
# Preprocess images and extract features for each condition
normal_images, normal_labels = preprocess_images(normal_dir, 'normal', target_width, target_height)
atelectasis_images, atelectasis_labels = preprocess_images(atelectasis_dir, 'atelectasis', target_width, target_height)
pneumonia_images, pneumonia_labels = preprocess_images(pneumonia_dir, 'pneumonia', target_width, target_height)

In [ ]:
# Combine data from all conditions
X = np.array(normal_images + atelectasis_images + pneumonia_images)
y = np.array(normal_labels + atelectasis_labels + pneumonia_labels)

In [ ]:
# Splitting data into training and testing sets (70:30 ratio)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
# Define AdaBoost classifier with Decision Tree as base estimator
base_estimator = DecisionTreeClassifier(max_depth=1)
adaboost_classifier = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=50, learning_rate=1.0, random_state=42)

In [ ]:
# Train AdaBoost classifier
adaboost_classifier.fit(X_train, y_train)

In [ ]:
# Evaluate the classifier
y_pred = adaboost_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
# Generate and print confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
# Manually input the confusion matrix values
# Replace these values with your actual confusion matrix values
conf_matrix_values = np.array('Fill the confusion matrix values printed here')

# Labels for the confusion matrix
labels = ['Normal', 'Atelectasis', 'Pneumonia']

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_values, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels,
            yticklabels=labels,
            annot_kws={"size": 16})  # Set font size for annotations
plt.title('Confusion Matrix', fontsize=18)  # Set title font size
plt.xlabel('Predicted Label', fontsize=14)  # Set x-axis label font size
plt.ylabel('True Label', fontsize=14)  # Set y-axis label font size
plt.show()

In [ ]:
# Generate and print classification report
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

In [ ]:
# Display images along with true and predicted outcomes in a 5x5 matrix
num_images = 10
fig = plt.figure(figsize=(15, 15))
for i in range(num_images):
    img = X_test[i].reshape(target_width, target_height)  # Reshape flattened image
    true_outcome = ''
    if y_test[i] == 0:
        true_outcome = 'True Outcome: Normal'
    elif y_test[i] == 1:
        true_outcome = 'True Outcome: Atelectasis'
    else:
        true_outcome = 'True Outcome: Pneumonia'

    predicted_outcome = ''
    if y_pred[i] == 0:
        predicted_outcome = 'Predicted Outcome: Normal'
    elif y_pred[i] == 1:
        predicted_outcome = 'Predicted Outcome: Atelectasis'
    else:
        predicted_outcome = 'Predicted Outcome: Pneumonia'

    ax = fig.add_subplot(5, 5, i+1)
    ax.imshow(img, cmap='gray')
    ax.set_title(true_outcome + '\n' + predicted_outcome)
    ax.axis('off')

plt.tight_layout()
plt.show()